In [26]:
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import floatpy.readers.padeops_reader as por

directory = '/projects/ShockInducedMix/ShearLayerData/production/Mc07/rr8/256x384x128/no_tkeb'
#directory = '/projects/ShockInducedMix/ShearLayerData/production/Mc12/rr1/256x384x128/'
filename_prefix = directory+'/shearlayer_'
reader = por.PadeopsReader(filename_prefix, periodic_dimensions=(True,False,True))
[Nx,Ny,Nz] = reader.domain_size
[x,y,z] = reader.readCoordinates()
yplot = y[0,:,0]

#ts = 34
#te = 60
xdir = 0;
zdir = 2;
print(reader.steps)

id_shift = 0#32-6 # for no_tkeb bc timing is offset

IOError: Unable to open file (unable to open file: name = '/projects/ShockInducedMix/ShearLayerData/production/Mc12/rr8/256x384x128/no_tkeb/shearlayer_coords.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [7]:
# Get thicknesses
fname = filename_prefix+'growth.dat'
dat = np.fromfile(fname,dtype=float, count=-1, sep=' ')
n = np.size(dat)

nstats=3
dat = np.reshape(dat,[n/nstats,nstats])
time = dat[:,0]
dtheta = dat[:,1]
domega = dat[:,2]
 
print("Thickness data available from {}-{}".format(time[0],time[np.size(time)-1]))

Thickness data available from 0.0-1180.0


In [ ]:
# Plot mean u, normalized
def mean_xz(q3D):
    tmp = np.mean(q3D,axis=zdir)
    return np.mean(tmp,axis=xdir)

plt.figure(figsize=(6,4),dpi=500)

ts = 55
te = 60
count = 0
unorm_sum = np.zeros(Ny)
for step in range(ts,te,5):
    reader.step = step
    step = step-id_shift
    u = reader.readData(('u'))
    u = np.array(u)
    ubar = mean_xz(u[0,:,:,:])
    U1 = ubar[Ny-1]
    U2 = ubar[0]
    du = U1-U2
    unorm = (ubar-U2)/(du)- 0.5
    tnorm = reader.time * du/dtheta[step]
    plt.plot(yplot/dtheta[step],unorm,alpha=0.5,label='t = {}'.format(tnorm))
    unorm_sum = unorm_sum + unorm
    count = count+1
    
plt.xlim([-20,20])
plt.xlabel('$\eta$')
plt.ylabel('$U(\eta)$')
plt.legend(bbox_to_anchor=(1., 1.))


In [23]:
# Plot R11,R22,R12 stress profiles
fig, ax = plt.subplots(1,3, figsize=(12,3), dpi=2000)

for step in range(ts,te,5):
    reader.step = step
    q = reader.readData(('u','v','w'))
    q = np.array(q)
    u = q[0,:,:,:]
    v = q[1,:,:,:]
    #w = q[2,:,:,:]
    ubar = mean_xz(u)
    vbar = mean_xz(v)
    #wbar = mean_xz(w)
    uprime1D = u - ubar[None,:,None];
    vprime1D = v - vbar[None,:,None];
    #wprime1D = w - wbar[None,:,None];
    
    uu = mean_xz(abs(uprime1D))
    vv = mean_xz(abs(vprime1D))
    #ww = mean_xz(abs(wprime1D))
    uv = mean_xz(np.sqrt(abs(uprime1D)*abs(vprime1D)))

    U1 = ubar[Ny-1]
    U2 = ubar[0]
    du = U1-U2
    
    step = step-id_shift
    ax[0].plot(yplot/dtheta[step],uu/du,alpha=0.5)
    ax[1].plot(yplot/dtheta[step],vv/du,alpha=0.5)
    ax[2].plot(yplot/dtheta[step],uv/du,alpha=0.5)

ax[0].set_title('$\sqrt{R_{11}}/\Delta U$')
ax[1].set_title('$\sqrt{R_{22}}/\Delta U$')
ax[2].set_title('$\sqrt{R_{12}}/\Delta U$')
ax[1].set_xlabel('$\eta$')

for a in ax:
    a.set_xlim([-20,20])
